# TSA Chapter 2: Parameter Estimation: Yule-Walker vs MLE

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch2/TSA_ch2_estimation/TSA_ch2_estimation.ipynb)

This notebook demonstrates:
- Comparison of Yule-Walker equations and Maximum Likelihood Estimation for AR(2) parameter estimation

In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from matplotlib.patches import Polygon
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    """Save chart as PDF and PNG."""
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
# Set seed

# Simulate AR(2)
n = 500
phi1_true, phi2_true = 0.6, -0.3
sigma_true = 1.0

epsilon = np.random.normal(0, sigma_true, n)
x = np.zeros(n)
for t in range(2, n):
    x[t] = phi1_true * x[t-1] + phi2_true * x[t-2] + epsilon[t]

# Estimation methods
# 1. Yule-Walker
model_yw = AutoReg(x, lags=2, old_names=False).fit()
phi1_yw = model_yw.params[1]
phi2_yw = model_yw.params[2]

# 2. MLE
model_mle = ARIMA(x, order=(2, 0, 0)).fit()
phi1_mle = model_mle.params[1]
phi2_mle = model_mle.params[2]

# Create comparison figure

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot 1: Series with fitted values
ax1 = axes[0]
ax1.plot(x[:100], 'b-', linewidth=0.8, alpha=0.8, label='Observed')
ax1.plot(model_mle.fittedvalues[:100], 'r--', linewidth=1.2, label='Fitted (MLE)')
ax1.set_title('AR(2) Series with Fitted Values', fontsize=12)
ax1.set_xlabel('Time')
ax1.set_ylabel('Value')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Parameter comparison
ax2 = axes[1]
methods = ['True', 'Yule-Walker', 'MLE']
phi1_vals = [phi1_true, phi1_yw, phi1_mle]
phi2_vals = [phi2_true, phi2_yw, phi2_mle]

x_pos = np.arange(len(methods))
width = 0.35

bars1 = ax2.bar(x_pos - width/2, phi1_vals, width, label=r'$\phi_1$', color='blue', alpha=0.7)
bars2 = ax2.bar(x_pos + width/2, phi2_vals, width, label=r'$\phi_2$', color='red', alpha=0.7)

ax2.set_ylabel('Parameter Value')
ax2.set_title('Parameter Estimates Comparison', fontsize=12)
ax2.set_xticks(x_pos)
ax2.set_xticklabels(methods)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
save_chart(fig, 'ch2_estimation_comparison')
plt.show()

print("Estimation Results:")
print(f"  True:        phi1={phi1_true:.4f}, phi2={phi2_true:.4f}")
print(f"  Yule-Walker: phi1={phi1_yw:.4f}, phi2={phi2_yw:.4f}")
print(f"  MLE:         phi1={phi1_mle:.4f}, phi2={phi2_mle:.4f}")